In [2]:
import sys
import os
os.environ['PROJ_DATA'] = "/pscratch/sd/p/plutzner/proj_data"
import xarray as xr
import numpy as np
import importlib as imp
import pandas as pd
import matplotlib.pyplot as plt
import cftime
import pickle
import cartopy.crs as ccrs
import json
import pickle
import gzip
import scipy
from scipy import stats
from cftime import DatetimeNoLeap
from datetime import datetime

In [12]:
# Load the two NetCDF files
ds1 = xr.open_dataset("/pscratch/sd/p/plutzner/E3SM/bigdata/ERA5/ERA5_1x1_daily_total_precipitation_1940-2023.nc")
ds2 = xr.open_dataset("/pscratch/sd/p/plutzner/E3SM/bigdata/ERA5/ERA5_1x1_skin_temperature_1940-2024.nc")

# Rename coordinates (modify if your names differ)
ds1 = ds1.rename({'longitude': 'lon', 'latitude': 'lat', 'valid_time': 'time'})
ds2 = ds2.rename({'longitude': 'lon', 'latitude': 'lat', 'valid_time': 'time'})


# Remove duplicates from ds2 (shouldn’t hurt to keep this)
_, index = np.unique(ds2['time'], return_index=True)
ds2 = ds2.isel(time=index)

# ds1 might still be hourly — resample to daily totals
ds1_daily = ds1.resample(time='1D').sum()

# Trim both to same time range
ds1_daily = ds1_daily.sel(time=slice("1940-01-01", "2023-12-31"))
ds2 = ds2.sel(time=slice("1940-01-01", "2023-12-31"))

# # Rename variables for clarity
# ds1_daily = ds1_daily.rename({'total_precipitation': 'precipitation'})
# ds2 = ds2.rename({'skin_temperature': 'temperature'})

# Align time coordinates explicitly
common_times = np.intersect1d(ds1_daily.time.values, ds2.time.values)
ds1_daily = ds1_daily.sel(time=common_times)
ds2 = ds2.sel(time=common_times)

# Merge
combined = xr.merge([ds1_daily, ds2])

In [13]:
# print(combined.skt)
# print(combined.tp)
# print(combined.time)
print(combined.dims)
print(combined.coords)
print(combined.var)

FrozenMappingWarningOnValuesAccess({'time': 30681, 'lat': 181, 'lon': 360})
Coordinates:
  * lon      (lon) float64 3kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat      (lat) float64 1kB 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
  * time     (time) datetime64[ns] 245kB 1940-01-01 1940-01-02 ... 2023-12-31
<bound method DatasetAggregations.var of <xarray.Dataset> Size: 16GB
Dimensions:  (time: 30681, lat: 181, lon: 360)
Coordinates:
  * lon      (lon) float64 3kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat      (lat) float64 1kB 90.0 89.0 88.0 87.0 ... -87.0 -88.0 -89.0 -90.0
  * time     (time) datetime64[ns] 245kB 1940-01-01 1940-01-02 ... 2023-12-31
Data variables:
    tp       (time, lat, lon) float32 8GB 0.003815 0.003815 ... 0.4701 0.4701
    skt      (time, lat, lon) float32 8GB ...
Attributes:
    CDI:                     Climate Data Interface version 2.0.5 (https://mp...
    Conventions:             CF-1.7
    institution:             European Cen

In [14]:
# Save the combined dataset to a new NetCDF file
combined.to_netcdf("/pscratch/sd/p/plutzner/E3SM/bigdata/ERA5/ERA5_1x1_input_vars_1940-2023.nc")


combined_input_file = '/pscratch/sd/p/plutzner/E3SM/bigdata/ERA5/ERA5_1x1_input_vars_1940-2023.nc'
era5_input = xr.open_dataset(combined_input_file)
era5_input.time

<xarray.DataArray 'time' (time: 30681)> Size: 245kB
array(['1940-01-01T00:00:00.000000000', '1940-01-02T00:00:00.000000000',
       '1940-01-03T00:00:00.000000000', ..., '2023-12-29T00:00:00.000000000',
       '2023-12-30T00:00:00.000000000', '2023-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 245kB 1940-01-01 1940-01-02 ... 2023-12-31
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         valid_time_bnds
    axis:           T

In [3]:
combined_input_file = '/pscratch/sd/p/plutzner/E3SM/bigdata/ERA5/ERA5_1x1_input_vars_1940-2023.nc'
era5_input = xr.open_dataset(combined_input_file)

In [10]:
print(era5_input.skt[:, 80, 80].isel(time = range(40)).values)

[299.3007  299.2514  299.21414 299.18008 299.15048 299.11005 299.07858
 298.99707 298.9644  298.95767 298.91962 298.8738  298.93103 298.98105
 298.9526  298.94388 298.95764 298.95505 299.0235  298.99637 298.98083
 299.022   299.07166 299.1211  299.1614  299.18625 299.2427  299.25092
 299.2667  299.28452 299.27438 299.26547 299.442   299.63574 299.61072
 299.68323 299.65942 299.4525  299.40775 299.39633]


In [12]:
# Look for nans in the tp and skt vars:
print(np.isnan(era5_input.tp).sum())
print(np.isnan(era5_input.skt).sum())

<xarray.DataArray 'tp' ()> Size: 8B
array(0)
<xarray.DataArray 'skt' ()> Size: 8B
array(0)
